# Import necessary libraries

In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
#Image processing
from skimage import measure
from skimage import filters
from skimage.feature import peak_local_max
from scipy import ndimage as ndi
from skimage import segmentation
from skimage.segmentation import watershed, clear_border
from skimage import measure, color
from scipy.optimize import curve_fit
from skimage.morphology import extrema
import matplotlib.patches as mpatches
import skimage as ski
import pandas as pd
# file management imports|
import os  ### only for count of images from dir, can be removed later
# model imports for deep learning 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
import keras
from keras import optimizers, regularizers
from tensorflow.keras.utils import to_categorical
# image processing imports
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,  img_to_array, load_img
from sklearn import metrics
#for confusion matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import itertools    
#for displaying images when predicting class
from PIL import Image, ImageOps
#for rounding up fitting model for steps_per_epoch
import math
from keras.models import load_model,Model
#Xplique
from xplique.attributions import GradientInput,GradCAM

# Preprocess and binarization

In [23]:
# Saturated contrast/brightness modification
def contrast_brightness_modify(img, alpha, beta):
    # Add bias and gain to an image with saturation arithmetics. 
    # Varialbes :
        # name_img : You need to put the name of the image file hear (png, jpg...)
        # alpha : You choose the good paramatre for the contrast control = multiple of each pixel's intensity
        # beta : You choose the value for the brightness control = offset intensity for each bit   
    new_img = img * alpha + beta
    new_img[new_img < 0] = 0
    new_img[new_img > 255] = 255
    return new_img.astype(np.uint8)
    
# Histogram equalization algorithm
# This function clip a percentage of the histogram from the bottom.
# User can add a clipping method from the top.
def automatic_brightness_and_contrast(image, clip_hist_percent=1,inverse_color=False):
    # This fuction can auto-adjust the brightness and contrast of the image using histogram equalization method and return 
    # the alpha (contrast) and the beta (brightness) coefficients of the image

    # Use inverse_color to make the back ground black and object white
    
    # The most important parameter of this function is clip_hist_percent.
    # This parameter can be exploited and added to main function if necessary. 

    # Convert to gray scale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    if inverse_color:
        gray = 255 - gray
        
    # Calculate grayscale histogram
    hist = cv2.calcHist([gray],[0],None,[256],[0,256])
    hist_size = len(hist)
    
    # Calculate cumulative distribution from the histogram
    accumulator = []
    accumulator.append(float(hist[0]))
    for index in range(1, hist_size):
        accumulator.append(accumulator[index -1] + float(hist[index]))
    
    # Locate points to clip
    maximum = accumulator[-1]
    clip_hist_percent *= (maximum/100.0)
    clip_hist_percent /= 2.0
    
    # Locate left cut
    minimum_gray = 0
    while accumulator[minimum_gray] < clip_hist_percent:
        minimum_gray += 1
    
    # Locate right cut
    maximum_gray = hist_size -1
    while accumulator[maximum_gray] >= (maximum - clip_hist_percent):
        maximum_gray -= 1
    
    # Calculate alpha and beta values
    alpha = 255 / (maximum_gray - minimum_gray)
    beta = -minimum_gray * alpha
    
    # Calculate new histogram with desired range and show histogram 
    new_hist = cv2.calcHist([gray],[0],None,[256],[minimum_gray,maximum_gray])
    plt.figure()
    plt.title('Gray level histogram')
    plt.xlabel('Pixel value')
    plt.ylabel('Number of pixels')
    plt.plot(hist)
    plt.plot(new_hist)
    plt.xlim([0,256])
    plt.show()

    auto_result = contrast_brightness_modify(gray, alpha=alpha, beta=beta)
    return (auto_result, alpha, beta)

# Gamma correction. Not yet utilized in this algorithm.
def adjust_gamma(image, gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

# Image process algorithm.
def image_treatment(name_img, inverse_color = False ,kernel_morpho = 5,open_iter=1,close_iter=1,clear_bder = False ):
    # This function take an image from the repertory and do the image processing, include:
        # Show image
        # Denoise the Gaussian noise
        # Control the contrast, the brightness and the color for some images
        # Show the denoised image
        # Do the morphological operation
        # Return the treated image
    # Varialbes :
        # inverse_color : Some image need to be converted their background to black one and we treat the NP in white
            # inverse_color = false (default : no need to convert)
            # inverse_color = true
    # This function returns the processed image and the alpha, beta coefficients. If the binary image doesn't correspond well to what we want 
    # to process, use the contrast_brightness_modify() function to adjust alpha and beta.
    img = cv2.imread(name_img)
   # Denoising image
    img1 = cv2.fastNlMeansDenoising(img , h=10 , templateWindowSize=7 , searchWindowSize=21)
    # Automatic contrast/birghtness adjustment
    gray,alpha,beta = automatic_brightness_and_contrast(img1,1,inverse_color)
    # Denoising again since the noise is amplified after correcting contrast and brightness
    gray = cv2.fastNlMeansDenoising(gray , h=10 , templateWindowSize=7 , searchWindowSize=21)
    # Gaussian blur
    gray = cv2.GaussianBlur(gray , (7,7),sigmaX=1,sigmaY=1)
    # Sharpen image
    sharpen_filter=np.array([[-0,-1,-0],
                 [-1,5,-1],
                [-0,-1,-0]])

    sharp_image=cv2.filter2D(gray,-1,sharpen_filter)
    # Threshold the image using Binary + Otsu method
    _, binary = cv2.threshold(sharp_image, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    # Filter kernel for morphological operation
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_morpho,kernel_morpho))
    
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=open_iter) # Opening morphology
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=close_iter) # CLosing morphology

    # Clear object in contact with border
    if clear_bder:
        binary = clear_border(binary)

    # Plotting result
    fig = plt.figure(figsize=(12, 12))
    
    gs = fig.add_gridspec(2,2)
    ax2 = fig.add_subplot(gs[1, 0])
    ax3 = fig.add_subplot(gs[1, 1])
    ax1 = fig.add_subplot(gs[0, :])
    
    ax1.imshow(img)
    ax1.axis('off')
    ax1.set_title("Original image",weight="bold")
    
    ax2.imshow(sharp_image,cmap='gray')
    ax2.axis('off')
    ax2.set_title("Processed image", weight="bold")
    
    ax3.imshow(binary,cmap='gray')
    ax3.axis('off')
    ax3.set_title("Binarized image",weight="bold")
   
    return binary,alpha,beta

# Image process algorithm.
# Same principle as the previous algorithm except that this function require manual adjustment of contrast and brightness
def image_treatment_manuel(name_img, inverse_color = False ,kernel_morpho = 5,open_iter=1,close_iter=1,clear_bder = False,alpha=1,beta=0 ):
    # This function take an image from the repertory and do the image processing, include:
        # Show image
        # Denoise the Gaussian noise
        # Control the contrast, the brightness and the color for some images
        # Show the denoised image
        # Do the morphology
        # Return the treated image
    # Varialbes :
        # inverse_color : Some image need to be converted their background to black one and we treat the NP in white
            # inverse_color = false (default : no need to convert)
            # inverse_color = true
    # This function returns the processed image and the alpha, beta coefficients. If the binary image doesn't correspond well to what we want 
    # to process, use the contrast_brightness_modify() function to adjust alpha and beta.
    # Convert to gray scale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if inverse_color : 
        gray = 255 - gray

    gray = cv2.fastNlMeansDenoising(gray , h=10 , templateWindowSize=7 , searchWindowSize=21)
    
    gray = contrast_brightness_modify(gray,alpha,beta)
    

    gray = cv2.fastNlMeansDenoising(gray , h=10 , templateWindowSize=7 , searchWindowSize=21)

    gaussian_blur = cv2.GaussianBlur(gray , (7,7),sigmaX=1,sigmaY=1)
    
    sharpen_filter=np.array([[-0,-1,-0],
                 [-1,5,-1],
                [-0,-1,-0]])

    sharp_image=cv2.filter2D(gray,-1,sharpen_filter)

    _, binary = cv2.threshold(sharp_image, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_morpho,kernel_morpho))
    
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=open_iter)
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=close_iter)
    
    if clear_bder:
        binary = clear_border(binary)
        
    fig = plt.figure(figsize=(12, 12))
    
    gs = fig.add_gridspec(2,2)
    ax2 = fig.add_subplot(gs[1, 0])
    ax3 = fig.add_subplot(gs[1, 1])
    ax1 = fig.add_subplot(gs[0, :])
    
    ax1.imshow(img)
    ax1.axis('off')
    ax1.set_title("Original image",weight="bold")
    
    ax2.imshow(sharp_image,cmap='gray')
    ax2.axis('off')
    ax2.set_title("Processed image", weight="bold")
    
    ax3.imshow(binary,cmap='gray')
    ax3.axis('off')
    ax3.set_title("Binarized image",weight="bold")
   
    return binary,alpha,beta

# Scale bar detection

In [24]:
# Scale bar detection 
# This fuction finds the object using Canny edge detection 
def detect_scale_bar(image_path,physical_length,inverse_color = False):
    # Variables :
        # image_path : the path directory to the image need to detect the scale bar
        # physical_length : length in nm
    # Image input + grayscale + inverse color if needed
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    if inverse_color : 
        gray_image = 255- gray_image
    # Gaussian blur
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    # Canny edge detection
    edges = cv2.Canny(blurred_image, 50, 150)
    # Contours detection based on edges detected
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    max_area = 0
    scale_bar_contour = None

    # Sort out the contour with the largest area 
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > max_area:
            max_area = area
            scale_bar_contour = contour
    # Draw detected scale bar
    if scale_bar_contour is not None:
        x, y, w, h = cv2.boundingRect(scale_bar_contour)
        cv2.rectangle(gray_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cut_image = np.copy(image)
        cut_image[y:y+h, x:x+w] = [0, 255, 0]
        plt.figure()
        fig, ax = plt.subplots(1,2,figsize=(15, 30))
        ax[0].imshow(image,cmap='gray')
        ax[0].axis('off') 
        ax[0].set_title("Scale bar image")

        handles = mpatches.Patch(color='lime', label='Detected scale bar')
        ax[1].legend(handles=[handles],bbox_to_anchor=( 1, -0.2),loc='upper right',
                      ncols=2, borderaxespad=0)
        ax[1].set_title("Detected scale bar")
        ax[1].imshow(cut_image,cmap='gray')
        ax[1].axis('off')        
    # Return the width of detected scale bar
        scale_bar_pixels = w
        scale_bar_ratio = physical_length/scale_bar_pixels
        return scale_bar_ratio
    else:
        return None

# Watershed Segmentation

### 1. Watershed with local extremum

In [13]:
# Watershed with local extremum
def NP_segmentation_local_max(name_img, min_distance,dist_max_threshold = 0.4,erode_iter=1,open_iter=0,kernel_size = 3) :
    # Variables:
        # name_img : The treated image from image_treatment function, you can choose anothe treated image with another method
        # min_distance : To find the local maximum point => just count the point with distance > min_distance
            # min_distance depend on the pixel distance between NPs in the image
            # We usually meet the case : min_d = 1..10 or min_d = 30..40
    # Filter kernel for morphological operation
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size,kernel_size))
    # Eroding morphology - Reduce isotropically object size
    # This operation helps a lot in revealing the line between object
    # But it can also clear out object with small size
    eroded = cv2.erode(name_img, kernel, iterations=erode_iter) 
    eroded = cv2.morphologyEx(eroded, cv2.MORPH_OPEN, kernel, iterations=open_iter)
    # Distance map transform with edt (Euclidean Distance Transform)
    distance = ndi.distance_transform_edt(eroded)
    # Normalizing distance map
    # Can only normalize pre-segmented zone
    ret, connected_regions = cv2.connectedComponents(eroded)
    normalized_distance_map = distance*0
    # Normalize each zone numerated by i
    for i in range(1,connected_regions.max()+1):
        # Extract zone i
        regions_distance = distance*(connected_regions==i)
        # Normalize this zone
        normalize_factor = 1/regions_distance.max()
        # Add normalized zone to the new distance map
        normalized_distance_map = normalized_distance_map + regions_distance*normalize_factor
    # Thresholdding the Distance map
    distance_map_erosed = ski.morphology.reconstruction(normalized_distance_map ,normalized_distance_map - normalized_distance_map.max()*dist_max_threshold
                                                    ,method = 'erosion')
    # Find Local maximum
    coords = peak_local_max(distance_map_erosed, min_distance = min_distance)
    # Seeds for Watershed segmentation
    mask = np.zeros(normalized_distance_map.shape, dtype=bool)
    mask[tuple(coords.T)] = True
    markers, _ = ndi.label(mask)
    # Watershed function
    labels_ws = watershed(-normalized_distance_map, markers, mask=name_img)
    # Color label segmented image
    img2=color.label2rgb(labels_ws,bg_label=0)
    # Plotting results
    fig, ax = plt.subplots(1,2,figsize=(12, 12))
    ax[0].imshow(name_img,cmap='gray')
    ax[0].axis('off') 
    ax[0].set_title("Binarized image")

    ax[1].set_title("Segmented image with labeled region")
    ax[1].imshow(img2)
    ax[1].axis('off')   
    return labels_ws

### 2. Watershed with Sure Foreground Extraction 

In [18]:
# Watershed segmentation with fore ground extraction
def NP_segmentation_fg_bg(name_img, dist_max_threshold = 0.4,erode_iter=1,open_iter=0,kernel_size = 3) :
    # Filter kernel for morphological operation
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size,kernel_size))
    # Eroding morphology - Reduce isotropically object size
    # This operation helps a lot in revealing the line between object
    # But it can also clear out object with small size
    eroded = cv2.erode(name_img, kernel, iterations=erode_iter)
    eroded = cv2.morphologyEx(eroded, cv2.MORPH_OPEN, kernel, iterations=open_iter)

    # Distance map transform with edt (Euclidean Distance Transform)
    distance = ndi.distance_transform_edt(eroded)
    # Normalizing distance map
    # Can only normalize pre-segmented zone
    ret, connected_regions = cv2.connectedComponents(eroded)
    normalized_distance_map = distance*0
    # Normalize each zone numerated by i
    for i in range(1,connected_regions.max()+1):
        # Extract zone i
        regions_distance = distance*(connected_regions==i)
        # Normalize this zone
        normalize_factor = 1/regions_distance.max()
        # Add normalized zone to the new distance map
        normalized_distance_map = normalized_distance_map + regions_distance*normalize_factor

    # Sure back ground using dilating morphology
    # Dilating morphology not exploited in this algorithm so the sure back ground is simply the processed image
    sure_bg = cv2.dilate(name_img,kernel, iterations= 0)
    # Sure for ground extraction by thresholdding normalized distance map
    ret2,sure_fg = cv2.threshold(normalized_distance_map,dist_max_threshold*normalized_distance_map.max(),255,cv2.THRESH_BINARY)
    # Convert float to unsigned integer
    sure_fg = np.uint8(sure_fg)
    # Unknown zone = Sure back ground - Sure fore ground
    unknown = cv2.subtract(sure_bg, sure_fg)
    # Seeds for Watershed segmentation
    mask_fg = np.zeros(sure_fg.shape, dtype=bool)
    mask_fg[sure_fg != 0] = True 
    markers, _ = ndi.label(mask_fg)
    # Watershed segmentation algorithm
    labels_ws = watershed(-normalized_distance_map, markers,mask=name_img)
    # Color labeling  seeds
    markers_plot=color.label2rgb(markers,bg_label=0)
    # Color labeling segmented image
    img2=color.label2rgb(labels_ws,bg_label=0)\
    # Plotting results
    fig = plt.figure(figsize=(12, 12))

    gs = fig.add_gridspec(2,2)
    ax2 = fig.add_subplot(gs[1, 0])
    ax3 = fig.add_subplot(gs[1, 1])
    ax1 = fig.add_subplot(gs[0, :])
    
    ax1.imshow(name_img,cmap='gray')
    ax1.axis('off')
    ax1.set_title("Binarized image",weight="bold")
    
    ax2.imshow(markers_plot)
    ax2.axis('off')
    ax2.set_title("Extracted Sure Foreground (Seeds)", weight="bold")
    
    ax3.imshow(img2)
    ax3.axis('off')
    ax3.set_title("Segmented image with labeled regions",weight="bold")

    return labels_ws

# Size Histogram and Calculation

### 1. Size histogram

In [6]:
# Size histogram plotting 
def size_histogram(labels_ws, pixel_to_nm, name_img,bins = 100) : 
    #
    # Variables:
        # labels_ws : image after segmentation processing (NP_segmentation)
        # pixel_to_nm : exchange rate between pixels and scale bar (detect_scale_bar)
        # name_img : the original image
    # Read image
    img = cv2.imread(name_img)
    # Measure segmented regions properties
    regions = measure.regionprops(labels_ws, intensity_image=img)
    # Image to plot result
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    labels_ws = labels_ws > 0
    back_ground = gray_image*0 +255
    img_label = gray_image*labels_ws
    img_label= img_label+back_ground
    # Insert sizes into an array
    i=0
    radius = np.array([])

    for regions_prop in regions :
        radius = np.append(radius,np.sqrt(regions_prop['Area']*pixel_to_nm**2/np.pi)) # Assume that np is circle
    # Plotting results
    fig, axs = plt.subplots(1,2,figsize=(21, 7))
    
    axs[0].imshow(img_label,cmap='gray')
    axs[0].axis('off')
    axs[0].set_title("Original Extracted Image")
    
    axs[1].hist(radius,bins = bins)
    axs[1].set_xlabel("Radius(nm)")
    axs[1].set_ylabel("Probability")
    axs[1].set_title("Size Histogram")
    axs[1].grid('on')
            
    return radius


### 2. Hsitogram dividision

In [9]:
def divide_histogram(radius, edge_radius) :
    #
    # Variables:
        # radius : The radius of each NP finded by radius_histogram
        # edge_radius : This parametre is needed to define different group of NPs, we define it after using radius_histogram
    
    radius1 = np.array([])
    for rad in radius:
         if rad < edge_radius:
             radius1 = np.append(radius1,rad)

    radius2 = np.array([])
    for rad in radius:
         if rad > edge_radius  :
             radius2 = np.append(radius2,rad)
    
    return (radius1 , radius2)

### 3. Gaussian Fit 

In [2]:
# Gaussian distribution function
def gaus(X,C,X_mean,sigma):
    return C*np.exp(-(X-X_mean)**2/(2*sigma**2))

# Gaussian curve fir algorithm
# In this approximation algorithm, we create the histogram with interval calculated from min and max value which can be mathematically incorrect
# User can optimize the histogram construction with 68–95–99.7 rule.
def Gaussian_fit(radius, bins):
    mean = np.mean(radius)
    variance = np.var(radius)
    # Histogram construction with intervals calculated from min and max value
    hist, bin_edges = np.histogram(radius,density='True',bins=bins, range = (0.8*np.min(radius),1.2*np.max(radius)))
    # Normalize histogram
    hist=hist/sum(hist)
    # Change histogram to discreted data points [x = 1/2(left value + right value);y = y histo]
    n = len(hist)
    x_hist=np.zeros((n),dtype=float) 
    for ii in range(n):
        x_hist[ii]=(bin_edges[ii+1]+bin_edges[ii])/2
    y_hist=hist
    # Initializing Gaussian fit parameters
    mean = sum(x_hist*y_hist)/sum(y_hist)                  
    sigma = sum(y_hist*(x_hist-mean)**2)/sum(y_hist) 
    # Gaussian curve fit
    param_optimised,param_covariance_matrix = curve_fit(gaus,x_hist,y_hist,p0=[max(y_hist),mean,sigma],maxfev=5000)
    return param_optimised,param_covariance_matrix,x_hist, y_hist
# Plotting Gaussian fit on Histogram constructed
def plot_Gaussian_fit(radius, bins):
    # Gaussian fitting
    param_optimised,param_covariance_matrix,x_hist,y_hist = Gaussian_fit(radius,bins)
    # Plotting result
    plt.figure()
    x_hist_2=np.linspace(np.min(x_hist),np.max(x_hist),500)
    plt.plot(x_hist_2,gaus(x_hist_2,*param_optimised),'r',label='Gaussian fit')
    plt.legend()
    # Calculted size
    str1 = "<r> =" + str(param_optimised[1]) + "+-"+str(np.sqrt(param_covariance_matrix[1,1]))+' nm '
    print(str1)

    weights = np.ones_like(radius) / len(radius)
    plt.hist(radius, weights=weights,bins = bins)
    plt.title(str1,weight = "bold")
    plt.xlabel("Radius(nm)")
    plt.ylabel("Probability")
    plt.grid('on')
    plt.show()


# Nanoparticles extraction

In [8]:
# Extract nanoparticles form original image
def extract_np(i,img, labels_ws,black_bg_color = False):
     # Variables:
        # i : Integer numerating the nanoparticle
        # img : Original image
        # labels_ws : Segmented mask
        # black_bg_color : indicate the color of the background (correctly correspond to model) 
    # Preparing extracted image
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    back_ground = gray_image*0 + 255
    back_ground_gray_image = back_ground*(labels_ws!=i)
    
    # Extract the i-th np mask
    extracted = labels_ws*(labels_ws==i)
    # Extract the i-th np from original image
    gray_image_1np = gray_image*(labels_ws==i) 
    gray_image_1np = gray_image_1np + back_ground_gray_image

    # Change type to unsigned int form float
    extracted =np.uint8(extracted)
    # Crop out the nanoparticle
    # Find the contour of np
    cnts = cv2.findContours(extracted, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    c = max(cnts, key=cv2.contourArea)
    # Locating left, right, top, bottom extremum of the np
    left_extreme = tuple(c[c[:, :, 0].argmin()][0])     
    right_extreme = tuple(c[c[:, :, 0].argmax()][0])
    top_extreme = tuple(c[c[:, :, 1].argmin()][0])
    bottom_extreme = tuple(c[c[:, :, 1].argmax()][0])
    # Crop out the np from located extremum
    extracted_np = gray_image_1np[top_extreme[1]:bottom_extreme[1],left_extreme[0]:right_extreme[0]]
    # Padding np image for the better classification (size image = 120%)
    h_old = extracted_np.shape[0]
    w_old = extracted_np.shape[1]
    h = math.floor(extracted_np.shape[0]*1.2)
    w = math.floor(extracted_np.shape[1]*1.2)
    calibrated_image  = np.full((h,w),255, dtype=np.uint8)
    x_center = (w - w_old) // 2
    y_center = (h - h_old) // 2
    calibrated_image[y_center:y_center+h_old, 
                     x_center:x_center+w_old] = extracted_np
    # Change back ground color if needed
    if black_bg_color:
        calibrated_image = 255 - calibrated_image
    # Convert array to image
    calibrated_image = Image.fromarray(calibrated_image)
    return calibrated_image    

# Extract binarized mask
# Same principle as the previous extraction
def extract_binary_np(i,img, labels_ws,black_bg_color = False):
    
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    back_ground = gray_image*0 +255
    back_ground_gray_image = back_ground*(labels_ws!=i)
    
    extracted = labels_ws*(labels_ws==i)
    
    binary_image_1np = extracted
    binary_image_1np = binary_image_1np + back_ground_gray_image
    
    extracted =np.uint8(extracted)
    
    cnts = cv2.findContours(extracted, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    c = max(cnts, key=cv2.contourArea)
    
    left_extreme = tuple(c[c[:, :, 0].argmin()][0])     
    right_extreme = tuple(c[c[:, :, 0].argmax()][0])
    top_extreme = tuple(c[c[:, :, 1].argmin()][0])
    bottom_extreme = tuple(c[c[:, :, 1].argmax()][0])
    
    extracted_np = binary_image_1np[top_extreme[1]:bottom_extreme[1],left_extreme[0]:right_extreme[0]]
    
    h_old = extracted_np.shape[0]
    w_old = extracted_np.shape[1]
    
    h = math.floor(extracted_np.shape[0]*1.2)
    w = math.floor(extracted_np.shape[1]*1.2)
    
    calibrated_image  = np.full((h,w),255, dtype=np.uint8)
    
    x_center = (w - w_old) // 2
    y_center = (h - h_old) // 2
    
    calibrated_image[y_center:y_center+h_old, 
                     x_center:x_center+w_old] =  extracted_np
    if black_bg_color:
        calibrated_image = 255 - calibrated_image

    calibrated_image = Image.fromarray(calibrated_image)
    
    return calibrated_image    


# Classification function and Explaination AI

In [2]:
# Classify an image with a model
def testing_image(img,model,target_size=(256,256),color_mode = 'L'):
    # Variables:
        # img : Input image
        # model : Model used to classify image
        # target_size : Target size of the original image, need to be matched with model's input
        # color_mode : 'L' for grayscale and 'RGB' for RGB, need to be matched with model's input
    # loading testing image with the target size for the image
    test_image = img.resize(target_size)
    # loading testing image with the target size for the image
    test_image = test_image.convert(mode=color_mode)
    # converts image into an array
    test_image = tf.keras.preprocessing.image.img_to_array(test_image)
    # expands array (from converted image) with a new dimension (for classifying values)
    test_image = np.expand_dims(test_image, axis = 0)
    test_image = tf.keras.applications.inception_v3.preprocess_input(test_image)
    # making prediction based on test_image and labeling it results
    result = model.predict(x = test_image)
    # printing predictions
    print(result)

    return result, test_image
    
# AI explainable via Xplique
# This Xplique function is only compatible for this whole process and notebook "Example of usage"
# In case of using Xplique for a specific image, please check out the examples of Xplique notebook
def show_xplique(model,img,label,total_label,alpha,method):
     # Variables:
        # img : Input image
        # model : Model used to classify image
        # label : explained label, usually equal np.argmax(result)
        # total_label : total number of classes
        # alpha : superimposed intensity of explained map onto original image
        # method : explainer method (GradientInput, Saliency,...)
    X = np.squeeze(img, axis = 0)
    explainer = method(model)
    Y = []
    labels = tf.keras.utils.to_categorical(label,total_label)
    Y.append(labels)
    explanations = explainer(img, Y)
    to_show = np.squeeze(explanations,axis=0)
    # Plotting result
    plt.figure()
    fig, ax = plt.subplots(figsize=(10,6))
    ax.imshow(X)
    plt.imshow(to_show, cmap="jet", alpha=alpha)
    plt.figure()


In [4]:
# Classification function
# This function classifies then returns a list of extracted labels correspond to different classes
def Classification(img_name,model,total_label,labels_ws,target_size=(256,256),color_mode='L',black_bg_color = False):
     # Variables:
        # img_name : Input image
        # model : Model used to classify image
        # total_label : total number of classes
        # labels_ws : Watershed label
        # target_size : Target size of the original image, need to be matched with model's input
        # color_mode : 'L' for grayscale and 'RGB' for RGB, need to be matched with model's input
        # black_bg_color : indicate the color of the background (correctly correspond to model) 
    # Convert to grayscale
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Intialize list for labels and classification result of each nanoparticle
    labels = []
    results = []
    # Classify all objects and put the results in a list
    for i in range(1,labels_ws.max()+1):
        np_i = extract_np(i,img, labels_ws,black_bg_color = black_bg_color)
        result, test_image = testing_image(np_i,model,target_size,color_mode)
        results.append(np.argmax(result))
    # Testing for each class
    for r in range(0,total_label):
        extracted = gray_image*0 +255
        labels_extracted = labels_ws*0
        index = 1
    # Testing for each object if their labeled class corresponds to the tested one
        for j in results:
            # If yes => add the object to a new label
            if j == r:
                extracted += gray_image*(labels_ws==index)
                labels_extracted += labels_ws*(labels_ws==index)
            # index indicate the number labeled of the selected object
            index += 1
        # Add the extracted label numerated by j to the label list after testing
        labels.append(labels_extracted)
        # Plotting the classified labels
        plt.figure()
        plt.title('Labeled class ' + str(r))
        plt.axis('off')
        plt.imshow(extracted,cmap='gray')
        plt.show()
    return labels